In [94]:
customer_collection = {}
import csv
valid_amount = []
def _amount_():
    try:
        flag = True
        amount = input()
        for a in amount:
            if a not in str(list(range(0,10))):
                flag = False
                raise TypeError('Amount must be numerical')
        if flag:
            valid_amount.append(float(amount))       
    except TypeError:
        _amount_()
class Account:
    def __init__(self, account_id,balance, interest = 0):

        if not isinstance(account_id,str):
            raise TypeError('Account ID must be string')    
        for id in account_id:
            if id not in str(list(range(0,10))):
                raise ValueError('Account ID must be within 0-9')
        # if not isinstance(balance,(int,float)):
        #     raise TypeError('Balance must be int or float') 
        self.account_id = account_id
        self.balance = float(balance)
        self.interest = interest

    def __str__(self):
        return f'{self.account_id},{round(float(self.balance),2)},{self.interest}'

    def _withdraw_(self):

        deducted_amount = float(input())

        try:
            self.balance -=  deducted_amount
            if self.balance < 0:
                raise ValueError()
        except ValueError:
            print('Cannot proceed your withdrawal')
            self.balance += deducted_amount
            
    def _deposit_(self):
        
        print('Amount: ',flush = True)
        _amount_()
        added_amount = valid_amount[-1]
        self.balance += added_amount
        
        
class Checking(Account):

    def __init__(self,account_id,balance,interest):
        super().__init__(account_id,balance,interest)
        self.interest = 0
    
    def __str__(self):
        return f'{self.account_id}  {round(float(self.balance),2)}'

class Savings(Account):
    def __init__(self,account_id,balance,interest):
        super().__init__(account_id,balance,interest)
        self.interest = 1
    
    def __str__(self):
        return f'{self.account_id} {round(float(self.balance),2)}'
    


In [95]:
customer_collection = {}

valid_name = []
def name_lookup():

    '''prompt user to put in only valid customer name and account type'''

    try:
        account_name = input('Customer name:')
        account_type = input('Checking or Savings or Credit')
        if account_name not in customer_collection.keys() or account_type not in ['Savings','savings','checking','Checking','credit',"Credit"]:
            raise ValueError('invalid name/account type')
        else:
            valid_name.append((account_name,account_type))
    except ValueError:
        name_lookup()

In [96]:
def import_account():
    global customer_collection
    step1 = input("which file do you want to access to?")
    with open(f'{step1}', 'r') as readfile:
        account = csv.DictReader(readfile)
        for row in account:
            checking_object = Checking(row['checking_id'], row['checking_balance'], '0')
            saving_balance = Savings(row['savings_id'], row['savings_balance'], '0')
            credit = Credit(row['credit_id'], row['credit_balance'], row['credit_limit'], '0')
            customer = Customer(row['username'], checking_object, saving_balance, credit)
            customer_collection[row['username']] = customer
            

In [97]:
def interface():

    import_ = import_account()

    '''display options and prompt user to choose'''

    print(' 1. View Customer\n',
            '2. Deposit\n',
            '3. Withdraw\n',
            '4. Credit Card Charge\n',
            '5. Credit Card Payment\n',
            '6. Exit\n')
    user_choice = input('Enter your 1/2/3/4/5/6')
    while user_choice not in str(list(range(1,8))):
        user_choice = input('Enter your 1/2/3/4/5/6')
    
    if user_choice == '1':
        print(customer_collection)
        for i in customer_collection.values():
            print(i)
        

    elif user_choice == '2':
        name_lookup()
        account_name, account_type = valid_name[-1]
        if account_type in ['Checking','checking']:
            customer_collection[account_name].checking._deposit_()
        elif account_type in ['Savings','savings']:
            customer_collection[account_name].savings._deposit_()
        
        
    ## Ghaith & Dallen: View customer and deposit HERE 
    ## Change if user_choise == '3  to elif 
    '''withdraw action'''

    if user_choice == '3':

        '''prompt the user to choose customer name and account type'''
        '''avoid typo/invalid name/account'''
                    
        name_lookup()
        account_name,account_type = valid_name[-1]

            
        '''proceed withdrawing by calling withdraw method for corresponding account type'''

        if account_type in ['Checking','checking']:
            customer_collection[account_name].checking._withdraw_()
        elif account_type in ['Savings','savings']:
            customer_collection[account_name].savings._withdraw_()

    if user_choice == '4':

        name_lookup()
        account_name,account_type = valid_name[-1]

        customer_collection[account_name].credit.__credit_card_charge__()

    if user_choice == '5':
        
        name_lookup()
        account_name,account_type = valid_name[-1]
        customer_collection[account_name].credit.credit_card_payment()

    if user_choice == '6': 

        exit_interface()

In [98]:
class Credit(Account):
    def __init__(self,account_id,balance,interest,credit_limit):
        super().__init__(account_id,balance)
        self.interest = interest
        self.credit_limit = credit_limit

    def __str__(self):
        return f"{self.account_id} {self.credit_limit}"
    
    def __credit_card_charge__(self):

        '''promt user to choose amount'''

        print('Amount: ', flush = True)
        _amount_()
        increased_amount = valid_amount[-1]

        '''prevent user to charge more than their limit'''
            
        if float(self.balance) + float(increased_amount) > float(self.credit_limit):
            return "you have gone over your credit limit"
        
        else:
            self.balance += increased_amount
            return f"balance is now {self.balance}"

    def credit_card_payment(self):
        
        name_lookup()
        account_name,account_type = valid_name[-1]
        print("Amount: ", flush = True)
        _amount_()
        amt = valid_amount[-1]
        
        if amt > customer_collection[account_name].savings.balance or amt > customer_collection[account_name].checking.balance:
            return f"Inputted amount higher than balance in {account_type}"
        elif amt > self.balance:
            return f"Inputted amount higher than outstanding balance"
        else:
            if account_type in ['Savings','savings']:
                customer_collection[account_name].savings.balance -= amt
                self.balance -= amt
            elif account_type in ['Checking',"Checking"]:
                customer_collection[account_name].savings.balance -= amt
                self.balance -= amt
                
        
        self.balance -= amt
        customer_collection[account_name].balance -= amt

        print(f"new balance : {self.balance}, current balance in {account_name} : {customer_collection[account_name].balance}")

In [99]:
class Customer:
    def __init__(self,account_id,checking,savings,credit):
        #all_data = []
        self.account_id = account_id

        self.checking = checking

        self.savings = savings

        self.credit = credit

    def __str__(self):
        return f"{self.account_id,self.checking.account_id,self.checking.balance,self.savings.account_id,self.savings.balance,self.credit.account_id,self.credit.balance,self.credit. credit_limit}"

In [104]:
def exit_interface():
    global customer_collection 
    paste_file = input("What file do you want to put your new changes? ")
    import csv 
    with open(f"{paste_file}", 'w') as readfile:
        writer = csv.writer(readfile)
        writer.writerow(['username','checking_id','checking_balance','savings_id','savings_balance','credit_id','credit_balance','credit_limit'])
        rows = []
        for keys in customer_collection.keys():
            str_customer = customer_collection[keys]
            rows.append(str(str_customer))
        writer.writerows(rows)


In [105]:
interface()

 1. View Customer
 2. Deposit
 3. Withdraw
 4. Credit Card Charge
 5. Credit Card Payment
 6. Exit

["('amohr', '1337', 43.0, '0666', 101.45, '1729', 5000.0, '0')", "('bbaggins', '2890', 15345.49, '2941', 15577483.0, '3021', 0.0, '0')", "('emusk', '0001', 21588737.58, '0002', 1000000000.0, '0003', 435678.58, '0')"]


In [31]:
def import_account():
    global customer_collection
    step1 = input("which file do you want to access to?")
    with open(f'{step1}', 'r') as readfile:
        account = csv.DictReader(readfile)
        for row in account:
            checking_object = Checking(row['checking_id'], row['checking_balance'], '0')
            saving_balance = Savings(row['savings_id'], row['savings_balance'], '0')
            credit = Credit(row['credit_id'], row['credit_balance'], row['credit_limit'], '0')
            customer = Customer(row['username'], checking_object, saving_balance, credit)
            customer_collection[row['username']] = customer

In [34]:
import_account()